In [ ]:
import matplotlib.pyplot as plt
import pylab as py
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms
import scipy.stats as stats
from sklearn import preprocessing
from numpy.random import seed
from numpy.random import rand
from numpy.random import randn
from numpy import mean
from numpy import var
from math import sqrt
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import math

Nacitanie datasetov na predspracovanie

In [ ]:
personal_data = pd.read_csv("Dataset/personal_train.csv")
other_data = pd.read_csv("Dataset/other_train.csv")

Funkcia ktora zavola vsetky potrebne upravovacie funkcie a vytvori finalny dataset a exportuje ho do .csv

In [ ]:
def prepare_data(personal_data, other_data):
    # remove unwanted column
    if 'Unnamed: 0' in personal_data:
        del personal_data['Unnamed: 0']
    if 'Unnamed: 0' in other_data:
        del other_data['Unnamed: 0']
    # merge datasets to create single large dataset with usefull data so it's easier to create graphs and analysis
    merged_medical_info_dataset = get_aggregated_data(other_data)
    usefull_dataset = personal_data.merge(merged_medical_info_dataset, on=['name', 'address'], how='outer')
    # replace NaN values with medians for numeric or most frequent values for categorical values
    usefull_dataset = remove_nans(usefull_dataset)
    # reduce number of features to improve performance
    usefull_dataset = feature_reduction(usefull_dataset)
    #backup values of class before scaling data
    backup_class = usefull_dataset[['age', 'class']]
    # detect and correct outliers
    usefull_dataset = outlier_detection(usefull_dataset)
    # scale and transform data
    usefull_dataset = transform(usefull_dataset)
    # remap class values
    usefull_dataset['class'] = backup_class['class'].values
    #export
    usefull_dataset.to_csv('Dataset/preprocessed_dataset.csv', index=False)
    return usefull_dataset


Funkcia na vytiahnutie a nacitanie agragovvanych udajov v datasete

In [ ]:
def get_aggregated_data(dataset):
    unique_medical_name_dataset = dataset.dropna(subset=['medical_info']).drop_duplicates('name')
    # create a dataset from 'medical_info' attribute transform it to JSON format and read as object to create dataframe
    medical_data_objects = []
    for index, record in unique_medical_name_dataset.iterrows():
        if isinstance(record['medical_info'], float):
            continue
        medical_object = json.loads(record['medical_info'].replace("\'", '\"').replace(':\"',':').replace('\",',',').replace('\"}','}'))
        medical_object['name'] = record['name']
        medical_data_objects.append(medical_object)
    medical_info_dataset = pd.DataFrame(medical_data_objects)
    merged_medical_info_dataset = unique_medical_name_dataset.merge(medical_info_dataset, on=['name'], how='outer').drop('medical_info', axis='columns')
    return merged_medical_info_dataset

Remove NaN values


In [ ]:
# Zbavovanie sa nan pomocou pipeline
def remove_nans(dataset):
    test = dataset
    # V stlpci class nan nenahrazdujeme medianom alebo najcastejsou hodnotou lebo by to mohlo dalej ovplivnovat vysledky
    # Takze radsej tie riadky ktore maju nan v stlpci class odstranujeme
    test = test[test['class'].isnull()==False]
    test.replace('?',np.NaN,inplace=True)

    # Vytiahneme si stlpce ktore obsahuju numericky a objektove hodnoty zvlast aby sme na kadzy druh vedeli aplikovat iny pipeline
    numeric_features = test.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = test.select_dtypes(include=['object']).columns

    num = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan,strategy='median'))])
    cat = Pipeline(steps=[('imputer', SimpleImputer(missing_values=np.nan,strategy='most_frequent'))])
    # na kazdy stlpec v tabulke aplikujeme pipeline tranformaciu pomocou column transformeru
    full = ColumnTransformer(
        transformers=[
            ('num',  num, numeric_features),
            ('cat', cat, categorical_features)])
    # pre opetovne zlepenie tabulky si spojime nazvy stlpcov lebo pipeline po fiit_tranformacii vyhodi tabulku s najprv numerickymi datami 
    # a potom s objektovymi
    columns = numeric_features.tolist() + categorical_features.tolist()
    dtype = {}
    for column in columns:
        dtype[column]=(test.dtypes.to_dict()[column])

    return pd.DataFrame(full.fit_transform(test), columns=columns,index=test.index).astype(dtype)

# Feature reduction

Funkcia na zredukovanie nepotrebnych atributov a vyselektovanie k najlepsich atributov ktore najviac kontribuuju k spravnej predpovedi vysledku

In [ ]:
def feature_reduction(dataset):
    test = dataset
    test = test[test['class'].isnull()==False]
    test.replace('?',np.NaN,inplace=True)
    
    numeric_features = test.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = test.select_dtypes(include=['object']).columns
    columns_to_remove = ['name', 'education-num', 'capital-gain', 'capital-loss', 'address', 'date_of_birth', 'occupation', 'marital-status','relationship', 'education', 'native-country', 'workclass', 'income', 'race']

    dataset['pregnant'] = dataset['pregnant'].apply(lambda value: 0 if re.search('T', value, re.IGNORECASE) else 1)
    # manualne odstranenie stlpcov ktore podla nas nemaju vplyv na vyslednu hodnotu (vacsinou kategoricke atributy)
    dataset_for_UST = dataset.drop(columns_to_remove, axis=1, errors='ignore')
    columns_to_remove.append('pregnant')
    categorical_features = categorical_features.drop(columns_to_remove, errors='ignore')

    dataset_for_UST = pd.get_dummies(dataset_for_UST, columns=categorical_features)

    # Feature Selection with Univariate Statistical Tests
    # pouzijeme statisticke tety aby sme vybrali K najlepsich atributov ktore najviac vplyvaju na vysledok

    UST_X = dataset_for_UST.drop('class', axis=1)
    UST_y = dataset_for_UST['class']
    # feature extraction
    test = SelectKBest(score_func=f_classif, k=13)
    fit = test.fit(UST_X, UST_y)
    # summarize scores
    #vidime ze niektore atributy maju miniaturny vplyv na vysledok priam az zanedbatelny tak ich vyhodime aby sme zrychlili vypocet modelu
    print(fit.scores_)
    features_columns = {}
    for i in range(len(fit.scores_)):
        features_columns[UST_X.columns[i]] = fit.scores_[i]
    columns_by_rank = {k: features_columns[k] for k in sorted(features_columns, key=features_columns.get, reverse=True)}
    columns_to_remove = list(columns_by_rank.keys())[13-(len(features_columns)):]
    return dataset_for_UST.drop(columns_to_remove, axis=1, errors='ignore')

# Odstranenie outlierov
zvolili sme metodu IRQ na detekciu outlierov a hodnoty ktore su outlieri nahradime krajnymi hodnotami

In [ ]:
def outlier_detection(dataset):
    for column in dataset.columns:
        Q1 = dataset[column].quantile(0.25)
        Q3 = dataset[column].quantile(0.75)

        IRQ = Q3 - Q1

        lower_bound = Q1 - (1.5*IRQ)
        upper_bound = Q3 + (1.5*IRQ)
        if(dataset[column].dtype == 'int64'):
            dataset[column] = dataset[column].apply(lambda val: math.floor(lower_bound) if val < lower_bound else math.floor(upper_bound) if val > upper_bound else val)
        else:
            dataset[column] = dataset[column].apply(lambda val: lower_bound if val < lower_bound else upper_bound if val > upper_bound else val)
        
        return dataset

# Data transofrmation

pouzijeme Robust scaler na transformaciu dat do podobnych rozsahov a power transformer na transformaciu rozdelenia dat blizsie ku normalovej distribucii

In [ ]:
def transform(dataset):
    scaler = RobustScaler()
    scaled = scaler.fit_transform(dataset)
    dataset = pd.DataFrame(scaled, columns=dataset.columns)

    power = PowerTransformer(method='yeo-johnson', standardize=True) 
    data_trans = power.fit_transform(dataset)
    return pd.DataFrame(data_trans, columns=dataset.columns)

In [ ]:
dataset = prepare_data(personal_data, other_data)
dataset.info()

Pozrieme sa na symetrickost a rozdelenie jednotlivych atributov v spracovanom datasete

In [ ]:
for column in dataset.columns:
    skewness = stats.skew(dataset[column])
    tab = 20 - len(column)
    if skewness <= 0.5 and skewness >= -0.5:
        print(column,' ' * tab, 'symmetrical    ', skewness)
    elif skewness < -0.5:
        print(column,' ' * tab, 'negative skew  ', skewness)
    else:
        print(column,' ' * tab, 'positive skew  ', skewness)

for column in dataset.columns:
    kurtosis = stats.kurtosis(dataset[column])
    tab = 20 - len(column)
    if kurtosis <= 3.5 and kurtosis >= 2.5:
        print(column,' ' * tab, 'symmetrical    ', skewness)
    elif skewness < 2.5:
        print(column,' ' * tab, 'in middle      ', skewness)
    else:
        print(column,' ' * tab, 'on outer       ', skewness)

# Prieskumna analyza
2 Datasety sme spojili do jedneho a odstranili sme duplicitne hodnoty a nulove hodnoty pre atribut class ktory je pre nas klucovy a nemozeme ho doplnit

Dalej sme zredukovali pocet atributov na take ktore najviac kontribuuju k vysledku a cyhodili take ktore pravdepodobne vobec nekontribuuju.
¨
Data sme taktiez transformovali aby sa viac podobali normalnej distribucii a pochadzali z podobnych rozsahov hodnot.

Tieto transformacie zmenili vyzor dat a v nasledujucej sekcii sa pozrieme na to ako sa zmenili distribucie a dalsie statisticke atributy dat.

In [ ]:
dataset.info()

In [ ]:
dataset['age'].describe()

In [ ]:
dataset['kurtosis_oxygen'].describe()

In [ ]:
dataset['std_oxygen'].describe()

In [ ]:
dataset['skewness_oxygen'].describe()

In [ ]:
dataset['mean_oxygen'].describe()

In [ ]:
dataset['std_glucose'].describe()

In [ ]:
dataset['kurtosis_glucose'].describe()

In [ ]:
dataset['skewness_glucose'].describe()

Mozeme si vsimnut ze vsetky atributy maju velmi pdobne rozsahy ale velmi rozdielne priemery avsak podobne standardne distribucie. toto je vysledok transformacii

In [ ]:
sns.pairplot(dataset, hue='class', kind='reg')

Kedze je cely dataset uz ako numericke hodnoty mozeme ho naraz vyhodnotit pre korelacia meddzi atributmi

In [ ]:
dataset.corr()

In [ ]:
dataset.cov()

### Parova analyza a zistovanie zavislosti veku(age) a sklonu glukozy(skewness_glucose)

pozrieme sa na to ako vyzeraju distribucie tychto atributov a nasledne ich zanalyzujeme

In [ ]:
sm.qqplot(dataset['age'], line='s')

In [ ]:
sm.qqplot(dataset['skewness_glucose'], line='s')

skewness_glucose z predoslej analyzy

![skewness_glucose](./skewness_glucose.png)

Atribut vek vyzera ako by mohol pochadzat z normalnej distribucie aj ked chvosty sa uz odklanaju

Atribut skewness glucose ma odlahle chvosty ale vdaka transformacii je to menej odchylene ako pred transformaciou

In [ ]:
sns.regplot(x=dataset['age'], y=dataset['skewness_glucose'])

In [ ]:
corr, _ = stats.pearsonr(dataset['age'], dataset['skewness_glucose']) 
print('Pearsons correlation: %.3f' % corr)

pre atributy age a skewness glucose sa nam corelacia takmer nezmenila predtym: 0.250

In [ ]:
from scipy.stats import kruskal

stat, p =kruskal(dataset['age'], dataset['skewness_glucose'])

print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')

In [ ]:
shapiro_test = stats.shapiro(dataset['age'])
print(shapiro_test)

# interpret
alpha = 0.05
if shapiro_test.pvalue > alpha:
    print('Normal distribution (fail to reject H0)')
else:
    print('Another distributions (reject H0)')

In [ ]:
shapiro_test = stats.shapiro(dataset['skewness_glucose'])
print(shapiro_test)

# interpret
alpha = 0.05
if shapiro_test.pvalue > alpha:
    print('Normal distribution (fail to reject H0)')
else:
    print('Another distributions (reject H0)')

Parova analyza mean_glucose a mean_oxygen

In [ ]:
corr, _ = stats.pearsonr(dataset['mean_oxygen'], dataset['mean_glucose']) 
print('Pearsons correlation: %.3f' % corr)

Pre tieto atributy sa nam zmenila korelacia viac a to z -0.322 na 0.030 moze to byt dosledkom odstranenia outlierov ktori tlacili korelaciu do negativnych hodnot

In [ ]:
from scipy.stats import kruskal

stat, p =kruskal(dataset['mean_oxygen'], dataset['mean_glucose'])

print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
alpha = 0.05
if p > alpha:
    print('Same distributions (fail to reject H0)')
else:
    print('Different distributions (reject H0)')

In [ ]:
shapiro_test = stats.shapiro(dataset['mean_oxygen'])
print(shapiro_test)

# interpret
alpha = 0.05
if shapiro_test.pvalue > alpha:
    print('Normal distribution (fail to reject H0)')
else:
    print('Another distributions (reject H0)')

In [ ]:
shapiro_test = stats.shapiro(dataset['mean_glucose'])
print(shapiro_test)

# interpret
alpha = 0.05
if shapiro_test.pvalue > alpha:
    print('Normal distribution (fail to reject H0)')
else:
    print('Another distributions (reject H0)')

In [ ]:
# testovanie predikcie class pomocou pipeline a troch classifierov z sklearn.ensemble
# najprv si rozdelime dataset na dve casti aby sme si pomocou train_test_split vedeli spravit trenovacie a testovacie data na predikciu
X = dataset.drop('class', axis=1)
y = dataset['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
classifiers = [
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier()
    ]
# nakolko data su uz spracovane nebolo treba v pipeline ich transformovat a vyhadzovat nan hodnoty
for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train, y_train)

    print(classifier)
    # print("model score: %.3f" % pipe.score(X_test, y_test))
    y_pred = pipe.predict(X_test)

    falseP = 0
    falseN = 0
    for i in range(len(y_pred)):
        if y_pred[i] != y_test.tolist()[i]:
            if y_pred[i] == 1:
                falseP += 1
            else: 
                falseN += 1
                
    print("False Positive: {}".format(falseP))
    print("False Negative: {}".format(falseN))
    print()